In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd

In [2]:
p = 0.4
a = 4
# use the y as directly from the house values
y = [0.5, 0.6, 0.3, 0.8]
x = cp.Variable(4)

equation = p*x + a*(cp.maximum(0,cp.exp(y-x)))
objective = cp.Minimize(cp.sum(equation))
constraints = [0 <= x]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
print(result)
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

6.1641360585955605
[2.8025854  2.90258539 2.60258531 3.10258536]
[3.36706060e-09 3.22350035e-09 3.68567251e-09 2.96184946e-09]


In [3]:
home = raw_input("Which Home?")
file_path = 'Year1/Home' + home + '_yr1.csv'

Which Home?1


In [4]:
#print(file_path)
dft = pd.read_csv(file_path, names=['Actual'])
#print(dft.head())
intervals = pd.date_range(start='12/1/2014', end='12/1/2015', freq='15min', closed=None)[0:-1]
dft.insert(0, 'Timestamp', intervals)
dft = dft.set_index('Timestamp')
print(dft)
#print(dft.index)
#print(dft['2014-12-1 00:00:00':])

                      Actual
Timestamp                   
2014-12-01 00:00:00  0.65018
2014-12-01 00:15:00  1.45400
2014-12-01 00:30:00  0.72971
2014-12-01 00:45:00  3.10750
2014-12-01 01:00:00  0.63572
2014-12-01 01:15:00  0.69720
2014-12-01 01:30:00  1.50610
2014-12-01 01:45:00  0.85858
2014-12-01 02:00:00  1.61320
2014-12-01 02:15:00  0.66075
2014-12-01 02:30:00  0.50952
2014-12-01 02:45:00  1.53310
2014-12-01 03:00:00  0.64315
2014-12-01 03:15:00  1.55420
2014-12-01 03:30:00  0.63596
2014-12-01 03:45:00  1.12640
2014-12-01 04:00:00  0.94811
2014-12-01 04:15:00  0.94247
2014-12-01 04:30:00  1.03720
2014-12-01 04:45:00  1.17440
2014-12-01 05:00:00  0.87564
2014-12-01 05:15:00  2.68930
2014-12-01 05:30:00  0.82416
2014-12-01 05:45:00  1.33590
2014-12-01 06:00:00  0.59462
2014-12-01 06:15:00  1.31450
2014-12-01 06:30:00  0.62424
2014-12-01 06:45:00  1.88760
2014-12-01 07:00:00  1.35300
2014-12-01 07:15:00  1.97930
...                      ...
2015-11-30 16:30:00  2.64140
2015-11-30 16:

In [6]:
from datetime import datetime, timedelta

In [7]:
year = raw_input("Enter the Year:")
month = raw_input("Enter the Month:")
date = raw_input("Enter the Date:")
hour = raw_input("Enter the Hour:")
minute = raw_input("Enter the Minute:")
date1 = datetime(int(year), int(month), int(date), int(hour), int(minute) , 0)
print(date1)

Enter the Year:2015
Enter the Month:11
Enter the Date:1
Enter the Hour:0
Enter the Minute:0
2015-11-01 00:00:00


In [8]:
# Find the end interval after 1 day
training_start_date = date1 - timedelta(days=7)
print(training_start_date)

2015-10-25 00:00:00


In [16]:
end_date = date1 + timedelta(days=1)
print(end_date)

2015-11-02 00:00:00


In [22]:
dataset = dft[training_start_date: date1 - timedelta(minutes=15)]
#y_train = dft[training_start_date: date1 - timedelta(minutes=15)]
#x_test = dft[i:j]
#y_test = dft[j:j+96]

print(dataset)

                      Actual
Timestamp                   
2015-10-25 00:00:00  0.84515
2015-10-25 00:15:00  0.61876
2015-10-25 00:30:00  1.45030
2015-10-25 00:45:00  0.37077
2015-10-25 01:00:00  1.38610
2015-10-25 01:15:00  0.65397
2015-10-25 01:30:00  1.06410
2015-10-25 01:45:00  0.57892
2015-10-25 02:00:00  1.22910
2015-10-25 02:15:00  0.67960
2015-10-25 02:30:00  1.33350
2015-10-25 02:45:00  0.58027
2015-10-25 03:00:00  1.42970
2015-10-25 03:15:00  0.68926
2015-10-25 03:30:00  1.05320
2015-10-25 03:45:00  1.02690
2015-10-25 04:00:00  0.47007
2015-10-25 04:15:00  1.42670
2015-10-25 04:30:00  0.58363
2015-10-25 04:45:00  1.15360
2015-10-25 05:00:00  0.59676
2015-10-25 05:15:00  1.09770
2015-10-25 05:30:00  0.57703
2015-10-25 05:45:00  1.16950
2015-10-25 06:00:00  0.94382
2015-10-25 06:15:00  0.84312
2015-10-25 06:30:00  1.12170
2015-10-25 06:45:00  0.53421
2015-10-25 07:00:00  1.05200
2015-10-25 07:15:00  0.67061
...                      ...
2015-10-31 16:30:00  0.72927
2015-10-31 16:

In [20]:
import itertools

In [25]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Examples of parameter combinations for Seasonal ARIMA
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


In [29]:
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [30]:
# find the optimal set of parameters that yields the best performance for our model
# Choose the one with the least resulting AIC value
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(dataset,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

ARIMA(0, 0, 0)x(0, 0, 0, 12)12 - AIC:2627.60186682
ARIMA(0, 0, 0)x(0, 0, 1, 12)12 - AIC:2434.22239738
ARIMA(0, 0, 0)x(0, 1, 0, 12)12 - AIC:2476.90698281
ARIMA(0, 0, 0)x(0, 1, 1, 12)12 - AIC:2042.17018674
ARIMA(0, 0, 0)x(1, 0, 0, 12)12 - AIC:2329.51640773
ARIMA(0, 0, 0)x(1, 0, 1, 12)12 - AIC:2078.49462408
ARIMA(0, 0, 0)x(1, 1, 0, 12)12 - AIC:2304.49348968
ARIMA(0, 0, 0)x(1, 1, 1, 12)12 - AIC:2044.17020982
ARIMA(0, 0, 1)x(0, 0, 0, 12)12 - AIC:2301.7692147
ARIMA(0, 0, 1)x(0, 0, 1, 12)12 - AIC:2199.55084827
ARIMA(0, 0, 1)x(0, 1, 0, 12)12 - AIC:2369.54274573
ARIMA(0, 0, 1)x(0, 1, 1, 12)12 - AIC:1927.04674248
ARIMA(0, 0, 1)x(1, 0, 0, 12)12 - AIC:2166.07212487
ARIMA(0, 0, 1)x(1, 0, 1, 12)12 - AIC:1964.10189633
ARIMA(0, 0, 1)x(1, 1, 0, 12)12 - AIC:2186.58016715
ARIMA(0, 0, 1)x(1, 1, 1, 12)12 - AIC:1929.04674736
ARIMA(0, 1, 0)x(0, 0, 0, 12)12 - AIC:2138.38762411
ARIMA(0, 1, 0)x(0, 0, 1, 12)12 - AIC:2109.21816273
ARIMA(0, 1, 0)x(0, 1, 0, 12)12 - AIC:2565.18686359
ARIMA(0, 1, 0)x(0, 1, 1, 12)12 -

In [31]:
print(file_path)
df = pd.read_csv(file_path, names=['Consumption'])
#print(df)
def datetime_range(start, end, delta):
    end += 4*delta
    current = start
    while current < end:
        yield current
        current += delta

dts = [dt for dt in 
       datetime_range(datetime(2014, 12, 1, 0), datetime(2015, 11 , 30 , 23), 
       timedelta(minutes=15))]
df['date/time'] = dts

Year1/Home1_yr1.csv


In [32]:
for i, row in df.iterrows():
#handling seasons
    df.at[i, 'Year'] = row[1].year
    df.at[i, 'Date'] = row[1].day
    df.at[i, 'Month'] = row[1].month
    df.at[i, 'Hour'] = row[1].hour
    df.at[i, 'Minute'] = row[1].minute
    if row[1].month in range(1,4) :
        df.at[i, 'Summer'] = 1
    elif row[1].month in range(4,7):
        df.at[i, 'Spring'] = 1
    elif row[1].month in range(7,10):
        df.at[i, 'Fall'] = 1
    elif row[1].month in range(10,13):
        df.at[i, 'Winter'] = 1
        
#handling peak hours   
    if row[1].hour in [0,1,2,3,4,5,6,7,20,21,22,23]:
        df.at[i, 'Peak Hour'] = 1
    else:
        df.at[i, 'Peak Hour'] = 0
        
#handling holidays & weekends
    if row[1].weekday() in [5,6] :
        df.at[i, 'Holiday'] = 1
    elif row[1].month == 10 and row[1].day == 2:
        df.at[i, 'Holiday'] = 1
    elif row[1].month == 1 and row[1].day == 26:
        df.at[i, 'Holiday'] = 1
    else:
        df.at[i, 'Holiday'] = 0

df['Spring'].fillna(0,inplace=True)
df['Fall'].fillna(0,inplace=True)
df['Summer'].fillna(0,inplace=True)
df['Winter'].fillna(0,inplace=True)

In [33]:
for i, row in df.iterrows():
    if row[1] == date1:
        break
x_train = df[['Year','Date','Month','Hour','Minute','Summer','Winter', 'Fall','Spring' , 'Holiday', 'Peak Hour']][i-672:i]
y_train = df[['Consumption']][i-672:i]
x_test = df[['Year','Date','Month','Hour','Minute','Summer','Winter', 'Fall','Spring' , 'Holiday', 'Peak Hour']] [i:96+i]
y_test = df[['Consumption']][i:96+i]

In [34]:
print(x_train)

         Year  Date  Month  Hour  Minute  Summer  Winter  Fall  Spring  \
31488  2015.0  25.0   10.0   0.0     0.0     0.0     1.0   0.0     0.0   
31489  2015.0  25.0   10.0   0.0    15.0     0.0     1.0   0.0     0.0   
31490  2015.0  25.0   10.0   0.0    30.0     0.0     1.0   0.0     0.0   
31491  2015.0  25.0   10.0   0.0    45.0     0.0     1.0   0.0     0.0   
31492  2015.0  25.0   10.0   1.0     0.0     0.0     1.0   0.0     0.0   
31493  2015.0  25.0   10.0   1.0    15.0     0.0     1.0   0.0     0.0   
31494  2015.0  25.0   10.0   1.0    30.0     0.0     1.0   0.0     0.0   
31495  2015.0  25.0   10.0   1.0    45.0     0.0     1.0   0.0     0.0   
31496  2015.0  25.0   10.0   2.0     0.0     0.0     1.0   0.0     0.0   
31497  2015.0  25.0   10.0   2.0    15.0     0.0     1.0   0.0     0.0   
31498  2015.0  25.0   10.0   2.0    30.0     0.0     1.0   0.0     0.0   
31499  2015.0  25.0   10.0   2.0    45.0     0.0     1.0   0.0     0.0   
31500  2015.0  25.0   10.0   3.0     0

In [35]:
print(y_train)

       Consumption
31488      0.84515
31489      0.61876
31490      1.45030
31491      0.37077
31492      1.38610
31493      0.65397
31494      1.06410
31495      0.57892
31496      1.22910
31497      0.67960
31498      1.33350
31499      0.58027
31500      1.42970
31501      0.68926
31502      1.05320
31503      1.02690
31504      0.47007
31505      1.42670
31506      0.58363
31507      1.15360
31508      0.59676
31509      1.09770
31510      0.57703
31511      1.16950
31512      0.94382
31513      0.84312
31514      1.12170
31515      0.53421
31516      1.05200
31517      0.67061
...            ...
32130      0.72927
32131      0.73064
32132      0.59854
32133      1.29840
32134      1.86110
32135      0.79159
32136      0.71204
32137      0.71036
32138      0.77162
32139      0.73239
32140      0.71010
32141      0.75897
32142      0.73297
32143      0.70385
32144      2.39010
32145      0.90559
32146      2.20200
32147      0.72889
32148      1.81230
32149      0.85572
32150      0